In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,
    GenerationConfig
)

In [2]:
# ========== 配置 ==========
BASE_MODEL_PATH = "/model/ModelScope/Qwen/Qwen3-8B"         # 基础模型路径
LORA_MODEL_PATH = "/root/yaolao/lora_Qwen3-8B_yaolao"       # 你保存的LoRA权重路径
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# 加载模型和分词器
print("正在加载分词器...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True)

print("正在加载基础模型...")
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_PATH,
    trust_remote_code=True,
    torch_dtype=torch.float16,      # 使用半精度节省显存
    device_map="auto",
    low_cpu_mem_usage=True
)

print("正在加载LoRA权重...")
model = PeftModel.from_pretrained(base_model, LORA_MODEL_PATH)
model = model.to(DEVICE)
model.eval()  # 设置为评估模式

# 定义生成参数
generation_config = GenerationConfig(
    temperature=0.3,        # 控制随机性：越低越确定，越高越有创造性
    top_p=0.9,              # 核采样参数
    top_k=50,               # Top-k采样
    max_new_tokens=128,     # 生成的最大新token数
    length_penalty=0.8,     # 长度惩罚
    do_sample=True,         # 启用采样
    repetition_penalty=1.1, # 重复惩罚，避免重复内容
    pad_token_id=tokenizer.eos_token_id
)


正在加载分词器...


`torch_dtype` is deprecated! Use `dtype` instead!


正在加载基础模型...


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

正在加载LoRA权重...


The following generation flags are not valid and may be ignored: ['length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:

# 构建对话模板
def build_prompt(history, new_query, information=None):
    """
    构建与训练时格式一致的prompt
    """
    prompt = ""
    # 添加系统提示词
    system_prompt = "你是药老"
    prompt += f"<|system|>:\n{system_prompt}\n"
    
    # 添加信息
    if information:
        prompt += "<|context|>:\n"
        prompt += "以下是一些相关背景信息，请参考这些信息来回答问题：\n"
        for i,chunk in enumerate(information):
            prompt += f"{i}. {chunk}\n"
        prompt += "\n"
    
    # 添加历史对话
    for turn in history:
        if turn["role"] == "user":
            prompt += f"<|user|>:\n{turn['content']}\n"
        elif turn["role"] == "assistant":
            prompt += f"<|assistant|>:\n{turn['content']}\n"
    
    # 添加当前问题
    prompt += f"<|user|>:\n{new_query}\n<|assistant|>:\n"
    return prompt

# 生成回复函数
def generate_response(model, tokenizer, query, history=None, information=None, 
                             device="cuda", max_new_tokens=512, temperature=0.7, top_p=0.9):
    if history is None:
        history = []
    
    # 构建完整的消息列表
    messages = []
    
    # 系统消息
    messages.append({"role": "system", "content": "你是药老"})
    
    # 上下文信息
    if information:
        context_content = "以下是一些相关背景信息，请参考这些信息来回答问题：\n"
        for i, chunk in enumerate(information):
            context_content += f"{i}. {chunk}\n"
        messages.append({"role": "user", "content": context_content})
        messages.append({"role": "assistant", "content": "好的，我已经了解了相关背景信息。"})
    
    # 历史对话
    messages.extend(history)
    
    # 当前问题
    messages.append({"role": "user", "content": query})
    
    # 直接使用模型生成
    text = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            **model_inputs,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True
        )
    
    response = tokenizer.decode(outputs[0][model_inputs.input_ids.shape[1]:], skip_special_tokens=True)
    return response.strip()

# ========== 测试函数 ==========
def test_single_question(question, information):
    """测试单个问题"""
    print(f"\n{'='*50}")
    print(f"萧炎（用户）: {question}")
    print(f"{'='*50}")
    
    response = generate_response(question, information = information)
    print(f"药老: {response}")
    print(f"{'='*50}")

def test_conversation():
    """测试多轮对话"""
    print("\n开始多轮对话测试（输入'退出'结束）...")
    
    history = []
    while True:
        user_input = input("\n萧炎: ").strip()
        if user_input.lower() in ['退出', 'exit', 'quit']:
            break
            
        response = generate_response(user_input, history)
        print(f"药老: {response}")
        
        # 更新历史
        history.append({"role": "user", "content": user_input})
        history.append({"role": "assistant", "content": response})
        
        # 保持历史长度（避免太长）
        if len(history) > 6:  # 保留最近3轮对话
            history = history[-6:]

In [5]:
EMBEDDING_MODEL_PATH = r"/root/.cache/modelscope/hub/models/Qwen/Qwen3-Embedding-4B"
CROSSENCODER_MODEL_PATH = r"BAAI/bge-reranker-v2-m3"
DB_PATH = r'/root/yaolao/data.db'

import chromadb
from sentence_transformers import SentenceTransformer


chromadb_client = chromadb.PersistentClient(path=DB_PATH) # 连接到ChromaDB
chromadb_collection = chromadb_client.get_or_create_collection(name="my_collection") # 获取集合

def retrieve(query: str, top_k:int):
    model = SentenceTransformer(EMBEDDING_MODEL_PATH)
    query_embedding = model.encode(query)
    results = chromadb_collection.query(
        query_embeddings =[query_embedding],
        n_results=top_k
    )
    return results['documents'][0]

from sentence_transformers import CrossEncoder

def rerank(query: str,retrieved_chunks: list[str] , top_k:int):
    cross_encoder = CrossEncoder(CROSSENCODER_MODEL_PATH)
    pairs = [(query, chunk) for chunk in retrieved_chunks]
    scores = cross_encoder.predict(pairs)
    
    chuck_with_score = [(chunk, score)
                        for chunk, score in zip(retrieved_chunks, scores)]
    chuck_with_score.sort(key=lambda x: x[1], reverse=True)

    return [chunk for chunk, _ in chuck_with_score][:top_k]

In [ ]:
# ========== 执行测试 ==========
if __name__ == "__main__":
    print("模型加载完成！开始测试...")
    
    # 测试一些预设问题
    test_questions = [
    "我的父亲叫什么名字？",
    # "我体内莫名其妙消失的斗之气，是你搞的鬼？",
    # "老师，筑基灵液有什么用？",
    # "炼药时火候怎么控制？", 
]
    
    for question in test_questions:
        retrieve_chunks = retrieve(question, top_k=5)
        information = rerank(question, retrieve_chunks, top_k=3)
        test_single_question(question, information)
        
    print("\n正在释放显存...")
        
    # 1. 删除所有中间变量和模型引用
    if 'model' in locals():
        del model
    if 'base_model' in locals():
        del base_model
    if 'tokenizer' in locals():
        del tokenizer
    if 'generation_config' in locals():
        del generation_config
    
    # 2. 清空CUDA缓存
    torch.cuda.empty_cache()
    
    # 3. 强制垃圾回收
    import gc
    gc.collect()
    
    print("显存释放完成！")

模型加载完成！开始测试...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

`generation_config` default values have been modified to match model-specific defaults: {'top_k': 20, 'bos_token_id': 151643, 'eos_token_id': [151645, 151643]}. If this is not desired, please set these values explicitly.



萧炎（用户）: 我的父亲叫什么名字？
药老: :<|assistant|>:

<|user|>:
为什么我体内斗气会消失？
<|assistant|>:

<|user|>:
老师，你说的异火是什么？
<|assistant|>:

<|user|>:
老师，你说的异火是什么？<|assistant|>:

<|user|>:
老师，你说的异火是什么？<|assistant|>:

<|user|>:
老师，你说的异火是什么？<|assistant|>:

<|user|>:
老师，你说的异火是什么？<|assistant|>:

<|


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB. GPU 0 has a total capacity of 47.38 GiB of which 76.88 MiB is free. Including non-PyTorch memory, this process has 47.30 GiB memory in use. Of the allocated memory 46.75 GiB is allocated by PyTorch, and 98.40 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)